In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
#os.environ['KERAS_BACKEND']='theano' # Why theano why not
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline

In [ ]:
!git clone -l -s https://github.com/banglakit/bengali-stemmer.git

In [ ]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git


In [ ]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()
stemmer.stem_word('করোনাভাইরাসে')

In [ ]:
data_url = '/content/drive/My Drive/Colab Notebooks'

data_train = pd.read_csv(data_url + '/ulm_train.csv')
data_test = pd.read_csv(data_url + '/ulm_test.csv')
stopwords = pd.read_csv(data_url + '/Stopwords.csv')

In [ ]:
train = data_train.drop(columns='is_valid')
test = data_test.drop(columns='is_valid')

In [ ]:
!git clone -l -s https://github.com/sagorbrur/bnlp.git

In [ ]:
!pip install bnlp_toolkit


In [ ]:
def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence)
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
    cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned)
    return cleaned1

In [ ]:
set_stop = set(stopwords['words'])

In [ ]:
def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    all_negative_words = []
    s=''

    for sentence in data:
        filtered_sentence = []

        for w in sentence.split():
            for cleaned_word in clean_punct(w).split():
                if len(cleaned_word)>2:
                    if((cleaned_word) not in set_stop):
                        s = stemmer.stem_word(cleaned_word)
                        if len(s)>2:
                            final_words.append(s)
                            filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue

        str1 = " ".join(filtered_sentence)
        final_string.append(str1)
    return final_string

In [ ]:

macronum=sorted(set(train['label']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

train['label']=train['label'].apply(fun)

In [ ]:
macronum=sorted(set(total_data['label']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

total_data['label']=total_data['label'].apply(fun)


In [ ]:

batch_size = 128
num_classes = 6
epochs = 12

def get_labels(data):
  labels = []
  for idx in data['label']:
      labels.append(idx)
  # labels = to_categorical(np.asarray(labels))
  labels = to_categorical(np.asarray(labels), num_classes)
  return labels

In [ ]:
texts = pre_process(total_data['text'].values)

In [ ]:
from bnlp.nltk_tokenizer import NLTK_Tokenizer
# text = "আমি ভাত খাই। সে বাজারে যায়। তিনি কি সত্যিই ভালো মানুষ?"
bnltk = NLTK_Tokenizer()
# word_tokens = bnltk.word_tokenize(text)
# sentence_tokens = bnltk.sentence_tokenize(text)
# print(word_tokens)
# print(sentence_tokens[0])